**NOTE**: Here we test the robustness of the STged with cell type pro proportion predicted from cell type deconvolution methods. We use results from different cell type deconvolution:
EnDecon,RCTD, SONAR, CARD,  Cell2location, or DWLS. 
##Set the size of the figures

In [1]:

#remove(list = ls())
#define the conda env

library(reticulate)
use_condaenv("ccnet", required = TRUE)

In [2]:
library(ggplot2)
library(scatterpie)
library(RColorBrewer)

In [3]:
# Source code
source("./code/STged.R")
source("./code/benchmark.R")

In [4]:
colors <- colorRampPalette(brewer.pal(9, "Set3"))(12)


# Load data the generated simulated data
## For the 50/100 um2
i = 1 for the 50 um2
i = 2 for the 50 um2

In [5]:
sim_data = readRDS("./realdata/MAOP/FN7_hash_mpoa_sim-29_ref01.Rds")

i = 2
patch_size = c(100,50)[i]
patch_size

mpoa = sim_data$mpoa_list[[i]]$`-0.29`


# load the referene from scRNA
sim_data = readRDS("./realdata/MAOP/refscRNA/scRNA_MPOA50k.RDS")
sc_exp = sim_data$scexp
sc_label = sim_data$sclabel

dim(sc_exp)
length(sc_label)

[1] 50

Loading required package: hash



hash-2.2.6.3 provided by Decision Patterns




[1]   134 29760

[1] 29760

sc_label[sc_label=="Mural"]="Pericytes"
table(sc_label)

## The spots location information

In [6]:
spot_raw = rownames(mpoa$patchGexp)
spots = paste0("spot",1:length(spot_raw))


split_strings <- strsplit(spot_raw, "_")
spot_loc <- do.call(rbind, lapply(split_strings, function(x) as.numeric(x)))
spot_exp <- t(mpoa$patchGexp)[,spot_raw ]
dim(spot_exp)

cell_type_freq =as.matrix(mpoa$cellTypeTable[spot_raw,]) 
cell_type_freq <- apply(cell_type_freq, 2, as.numeric)  
cell_type_prop = sweep(cell_type_freq, 1, rowSums(cell_type_freq), FUN = "/")

cell_type = colnames(cell_type_prop)
cell_type


rownames(cell_type_prop) = rownames(spot_loc)  = spots
colnames(spot_exp) = spots

rownames(cell_type_prop) = rownames(spot_loc)  = spots
colnames(spot_exp) = spots


[1]  135 1141

[1] "Astrocyte"   "Endothelial" "Ependymal"   "Excitatory"  "Inhibitory" 
[6] "Microglia"   "OD Immature" "OD Mature"   "Pericytes"

## TRUE cell type-specific gene expression for each cell type

In [7]:
genes = rownames(spot_exp)  
TRUE_F = vector("list", length(cell_type))  
names(TRUE_F) = cell_type  

spot_cell_type_exp = mpoa$avgPatchGexp

for(i in 1:length(cell_type)){
  
  temp_mat = matrix(0, nrow = length(spots), ncol = length(genes))
  colnames(temp_mat) = genes
  rownames(temp_mat) = spots
  
  for(j in 1:length(spots)){
    temp = spot_cell_type_exp[[j]]
    
    
    if (cell_type[i] %in% colnames(temp)) {
      temp_mat[j,] = temp[,cell_type[i]]
    } else {
      
      temp_mat[j,] = 0
    }
  }
  
  
  TRUE_F[[i]] = t(temp_mat)
}



In [8]:

res_decon_ct = readRDS("./realdata/MAOP/Endeprop/merfish_Results.Deconv_50um.RDS")


**Note**: There are three steps in STged. We here give a small example to show how to run the code step by step.

## Step 1: clear data

In [9]:
clean.only = FALSE
depthscale  = 1e6
datax = data_process(sc_exp = sc_exp,   sc_label = sc_label, 
                     spot_exp = spot_exp,  spot_loc = spot_loc,
                     depthscale = depthscale,  gene_det_in_min_cells_per = 0, 
                     expression_threshold = 0,
                     nUMI =  10, verbose = FALSE, clean.only = clean.only)

## Winsorize expression values to prevent outliers  
truncate = FALSE
if(truncate){
  qt = 0.05
  datax$sc_exp  =  winsorize(x =  datax$sc_exp, qt = qt)
  datax$spot_exp  =  winsorize(x =  datax$spot_exp, qt = qt)
}


In [10]:
for(i in 1:length(cell_type)){
  TRUE_F[[i]] = TRUE_F[[i]][,colnames(datax$spot_exp )]
  
}
trueF = res_library_size_normaization(estres = TRUE_F, depthscale = depthscale)


## Step 2: construct spatial correlation structures

In [11]:
cat("Construct spatial correlation", "\n")
L.mat = dis_weight(spot_loc = datax$spot_loc, spot_exp = datax$spot_exp, k = 6, 
                   quantile_prob_bandwidth = 1/3, method = "Hex", 
                   coord_type = "grid")

Construct spatial correlation 


## Step 3: construct reference gene matrix

In [12]:

cat("Construct reference gene matrix", "\n")

ref_exp = create_group_exp(sc_exp = datax$sc_exp, sc_label = datax$sc_label)


ref_exp = ref_exp[rownames(datax$spot_exp), cell_type]
colnames(ref_exp )

beta.type = cell_type_prop[colnames(datax$spot_exp),]

Construct reference gene matrix 


[1] "Astrocyte"   "Endothelial" "Ependymal"   "Excitatory"  "Inhibitory" 
[6] "Microglia"   "OD Immature" "OD Mature"   "Pericytes"

In [13]:
p = nrow(datax$spot_exp)
p 
K = ncol(beta.type)
beta.ind.temp = beta.type>0.05
beta.ind =  matrix(list(), K ,1)
for (i in 1:K){
  
  beta.ind[[i]] =  matrix( rep(beta.ind.temp[,i],p), nrow = p, byrow = TRUE)
}


[1] 134

## Step 4: run the main model
### Step 4-1: run the main model with ESTIMATED cell type proportion

In [ ]:
### run code with parameters selection
methods = names(res_decon_ct)
bench_celltype_STged = matrix(list(), length(methods) ,1)
for(ii in 1: length(methods) ){
  
  cat("Run the STged", methods[ii], "\n")
  beta.type.bench = res_decon_ct[[ii]][colnames(datax$spot_exp),]
  start_time <- Sys.time()
  stged.est = MUR.STged(srt_exp = datax$spot_exp, ref_exp = ref_exp, 
                        beta.type = beta.type.bench,   W = L.mat$dis_weight, cutoff = 0.05, 
                        lambda1 = NULL, lambda2 = NULL,
                        epsilon = 1e-5,  maxiter = 200)
  
  end_time <- Sys.time()
  bench_celltype_STged[[ii]] = stged.est$V.hat
  cat("Run time of STged", end_time - start_time,"\n") 
  
}

files = paste0("./results/MPOA/STged_est_spot_decon_Use_tureprop_tuning_est",patch_size,"um2.RDS")
saveRDS(bench_celltype_STged, file =files)
